In [40]:
from imports import *
from tools import *

msun = units.Msun.to('g')
pc2cm = units.pc.to('cm')

In [41]:
root_dir = '/Users/abarnes/Dropbox/work/Projects/NOEMA/data/CloudH_continuum/'
input_file = '%s/cloudH_3mm_cont.fits' % root_dir

output_dir = './dendro/'
input_dir = './dendro/'

cloud = 'CloudH'
distance = 2.9e3

In [51]:
print(input_file, cloud)

index_map = fits.getdata('%s/%s_index_map.fits' %(input_dir, cloud))
props = QTable.read('%s/%s_props.fits' %(input_dir, cloud))
hdu = fits.open(input_file)[0]
header = hdu.header

bave = np.array([header['BMAJ'], header['BMIN']])
bave = bave*3600
freq = header['RESTFREQ']

temp = 12 # Based on ~ average from Sokolov et al. 2017 NH3 data... 

Nh2_mean = column_density(props['mean_flux'].value, bave, freq, temp)
Nh2_peak = column_density(props['max_flux'].value, bave, freq, temp)

Mass = mass_calc(props['flux'].value, freq, temp, distance, gas2dust=141)
Mass_bgsub = mass_calc(props['flux_bgsub'].value, freq, temp, distance, gas2dust=141)

radius_pc = props['radius'].value * (distance/206265)
radiusdeconv_pc = props['radius_deconv'].value * (distance/206265)

radiuseff_pc = props['radius_eff'].value * (distance/206265)
radiuseffdeconv_pc = props['radius_eff_deconv'].value * (distance/206265)

n_h2 = number_density_sphere_pc(Mass, radiuseff_pc)
n_h2_bgsub = number_density_sphere_pc(Mass_bgsub, radiuseff_pc)

rho = get_rho(Mass, radiuseff_pc)
rho_bgsub = get_rho(Mass_bgsub, radiuseff_pc)

tff = tff_spherical(n_h2)
tff_bgsub = tff_spherical(n_h2_bgsub)

props.add_column(temp *au.K, name='temp')

props.add_column(np.array([distance] *len(radius_pc)) *au.pc, name='distance_pc')

props.add_column(radius_pc *au.pc, name='radius_pc')
props.add_column(radiusdeconv_pc *au.pc, name='radiusdeconv_pc')
props.add_column(radiuseff_pc *au.pc, name='radius_eff_pc')
props.add_column(radiuseffdeconv_pc *au.pc, name='radius_eff_deconv_pc')

props.add_column(Nh2_mean *au.cm**-2, name='columndensity_mean')
props.add_column(Nh2_peak *au.cm**-2, name='columndensity_peak')

props.add_column(Mass *au.Msun, name='mass')
props.add_column(Mass_bgsub *au.Msun, name='mass_bgsub')
props.add_column(n_h2 *au.cm**-3, name='number_density') 
props.add_column(n_h2_bgsub *au.cm**-3, name='number_density_bgsub')

props.add_column(rho, name='volume_density') 
props.add_column(rho_bgsub, name='volume_density_bgsub')

props.add_column(tff *au.Myr, name='tff') 
props.add_column(tff_bgsub *au.Myr, name='tff_bgsub') 

props.write('%s/%s_physprop.fits' %(output_dir, cloud), overwrite=True)

/Users/abarnes/Dropbox/work/Projects/NOEMA/data/CloudH_continuum//cloudH_3mm_cont.fits CloudH


In [52]:
props['radius_eff_deconv']

<MaskedQuantity [6.14126102, 3.68821161, 2.92262451, 2.22726669,
                 1.27217697, 1.43113812, 1.60417335,        ———,
                 1.15410263, 1.39737701, 0.94985832, 1.194758  ,
                        ———, 0.48350211,        ———,        ———] arcsec>

In [53]:
props['radius_eff_deconv'].to('arcsec'), props['radius_eff_deconv_pc'].to('pc')

(<MaskedQuantity [6.14126102, 3.68821161, 2.92262451, 2.22726669,
                  1.27217697, 1.43113812, 1.60417335,        ———,
                  1.15410263, 1.39737701, 0.94985832, 1.194758  ,
                         ———, 0.48350211,        ———,        ———] arcsec>,
 <MaskedQuantity [0.08634357, 0.05185472, 0.04109088, 0.03131444,
                  0.01788628, 0.02012121, 0.02255401,        ———,
                  0.0162262 , 0.01964654, 0.01335461, 0.0167978 ,
                         ———, 0.00679784,        ———,        ———] pc>)

In [54]:
props['mass']

<Quantity [19.27544455, 16.21893493, 14.23632284, 10.58974426,  5.82460553,
            4.49713769,  4.32210034,  4.20852427,  4.05240365,  3.82103733,
            3.27612061,  2.96492002,  2.81733954,  2.13246208,  1.26319816,
            1.05668721] solMass>

In [55]:
props['mass_bgsub']

<Quantity [4.91048729, 9.98017122, 6.44215977, 7.25118851, 2.07365285,
           2.16128784, 1.78136479, 0.8866621 , 1.94203002, 1.52012701,
           1.04918826, 0.84023855, 0.28975739, 0.40994757, 0.24952076,
           0.18722933] solMass>

In [56]:
props['flux_bgsub'].to('mJy')/props['flux'].to('mJy')

<Quantity [0.25475352, 0.61534073, 0.45251571, 0.68473689, 0.35601602,
           0.48059188, 0.41215258, 0.21068242, 0.47922917, 0.39783098,
           0.32025325, 0.28339333, 0.10284788, 0.19224143, 0.19753097,
           0.1771852 ]>

In [57]:
props['number_density_bgsub']/1e6

<Quantity [0.02209026, 0.15710681, 0.16390624, 0.29392182, 0.15669356,
           0.14834881, 0.10956944, 0.16692986, 0.1570234 , 0.10653671,
           0.09447107, 0.06639956, 0.08286592, 0.04438908, 0.07331387,
           0.05654663] 1 / cm3>

In [58]:
props['tff_bgsub'].to('yr')

<Quantity [207706.35972339,  77884.79426338,  76252.21144204,
            56942.203191  ,  77987.42914767,  80150.85245975,
            93262.15934347,  75558.47219179,  77905.47756111,
            94580.26790694, 100438.62675694, 119802.97294172,
           107241.33444006, 146525.13338388, 114013.72127938,
           129821.56078116] yr>